In [1]:
from glob import glob
import pandas as pd
import os
import numpy as np
import tensorflow as tf

c:\Users\huang\miniconda3\envs\gpu-ml-base\Lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
c:\Users\huang\miniconda3\envs\gpu-ml-base\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.6 when it was built against 1.14.5, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from glob import glob
import os

class TCRDataLoader:
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.aa_dict = None
        self.v_dict = None
        self.j_dict = None
        self.label_map = None
        self.max_length = None
        self.column_mapping = None
        
    def detect_column_mapping(self, df):
        """Automatically detect and map column names to standard names"""
        # Define possible column name variations
        column_variants = {
            'sequence': ['aminoAcid', 'beta', 'sequence', 'cdr3', 'CDR3', 'Amino'],
            'v_gene': ['v_beta', 'vGeneName', 'v_gene', 'V_gene', 'TRBV'],
            'j_gene': ['j_beta', 'jGeneName', 'j_gene', 'J_gene', 'TRBJ'],
            'counts': ['counts', 'count', 'count (templates/reads)', 'templates', 'reads', 'frequency']
        }
        
        mapping = {}
        available_columns = df.columns.tolist()
        
        for standard_name, variants in column_variants.items():
            found_column = None
            for variant in variants:
                if variant in available_columns:
                    found_column = variant
                    break
            
            if found_column:
                mapping[standard_name] = found_column
            else:
                if standard_name in ['sequence']:  # Required columns
                    print(f"Warning: No column found for {standard_name}")
                    print(f"Available columns: {available_columns}")
                    print(f"Expected variants: {variants}")
        
        self.column_mapping = mapping
        return mapping
    
    def load_files(self):
        df_list = []
        files = glob(self.data_dir + "/*/*.tsv")
        
        for file_path in files:
            df = pd.read_csv(file_path, sep='\t')
            
            # Extract folder name from file path
            folder_name = os.path.basename(os.path.dirname(file_path))
            
            # Split folder name to get Antigen
            parts = folder_name.split('-')
            antigen = parts[-1]  # Get last part (the antigen)
            
            # Add antigen column to dataframe
            df['Antigen'] = antigen
            
            df_list.append(df)
        
        combined_df = pd.concat(df_list, ignore_index=True)
        
        # Detect column mapping from the combined dataframe
        self.detect_column_mapping(combined_df)
        
        return combined_df
    
    def get_standardized_data(self, df):
        """Get data using the detected column mapping"""
        if not self.column_mapping:
            raise ValueError("Column mapping not detected. Run load_files() first.")
        
        standardized = {}
        
        # Get sequence data
        if 'sequence' in self.column_mapping:
            standardized['sequence'] = df[self.column_mapping['sequence']]
        else:
            raise ValueError("No sequence column found")
        
        # Get V gene data
        if 'v_gene' in self.column_mapping:
            standardized['v_gene'] = df[self.column_mapping['v_gene']]
        else:
            print("Warning: No V gene column found, using None")
            standardized['v_gene'] = pd.Series([None] * len(df))
        
        # Get J gene data
        if 'j_gene' in self.column_mapping:
            standardized['j_gene'] = df[self.column_mapping['j_gene']]
        else:
            print("Warning: No J gene column found, using None")
            standardized['j_gene'] = pd.Series([None] * len(df))
        
        # Get counts data
        if 'counts' in self.column_mapping:
            standardized['counts'] = df[self.column_mapping['counts']]
        else:
            print("Warning: No counts column found, using 1")
            standardized['counts'] = pd.Series([1] * len(df))
        
        return standardized
    
    def build_vocabulary(self, df):
        # Get standardized data
        std_data = self.get_standardized_data(df)
        
        # Amino acids
        all_sequences = std_data['sequence'].dropna()
        giant_string = ''.join(all_sequences)
        unique_letters = sorted(set(giant_string))
        self.aa_dict = {}
        for i, letter in enumerate(unique_letters):
            self.aa_dict[letter] = i+1
        
        # V genes
        v_genes = std_data['v_gene'].dropna().unique()
        v_genes = sorted(v_genes)
        self.v_dict = {}
        for i, gene in enumerate(v_genes):
            self.v_dict[gene] = i + 1

        # J genes  
        j_genes = std_data['j_gene'].dropna().unique()
        j_genes = sorted(j_genes)
        self.j_dict = {}
        for i, gene in enumerate(j_genes):
            self.j_dict[gene] = i + 1

        # Labels
        antigens = df['Antigen'].unique()
        self.label_map = {}
        for i, antigen in enumerate(sorted(antigens)):
            self.label_map[antigen] = i

        # Max length
        sequence_lengths = [len(seq) for seq in std_data['sequence'].dropna()]
        self.max_length = max(sequence_lengths) if sequence_lengths else 0
    
    def encode_sequences(self, sequences):
        encoded_aa = []
        for sequence in sequences:
            sequence_numbers = []
            for letter in sequence:
                number = self.aa_dict.get(letter, 0)
                sequence_numbers.append(number)
            # Add padding zeros if sequence is shorter than max_length
            while len(sequence_numbers) < self.max_length:
                sequence_numbers.append(0)
            encoded_aa.append(sequence_numbers)
        return encoded_aa
    
    def encode_v_genes(self, v_genes):
        encoded_v = []
        for v_gene in v_genes:
            encoded_v.append(self.v_dict.get(v_gene, 0))
        return encoded_v
    
    def encode_j_genes(self, j_genes):
        encoded_j = []
        for j_gene in j_genes:
            encoded_j.append(self.j_dict.get(j_gene, 0))
        return encoded_j
    
    def load_and_encode_data(self, batch_size=100, shuffle=True):
        # 1. Load files
        df = self.load_files()
        
        # 2. Build vocabularies
        self.build_vocabulary(df)
        
        # 3. Get standardized data
        std_data = self.get_standardized_data(df)
        
        # 4. Filter out rows with missing amino acid sequences
        valid_rows = std_data['sequence'].notna()  # Use standardized column name
        df_valid = df[valid_rows].copy()
        std_data_valid = {k: v[valid_rows] for k, v in std_data.items()}
        
        # 5. Get the data you want to encode (now all valid)
        sequences = std_data_valid['sequence'].values
        v_genes = std_data_valid['v_gene'].fillna('UNK').values
        j_genes = std_data_valid['j_gene'].fillna('UNK').values
        labels = [self.label_map[antigen] for antigen in df_valid['Antigen']]
        
        # 6. Encode everything
        X_sequences = self.encode_sequences(sequences)
        X_v_genes = self.encode_v_genes(v_genes)
        X_j_genes = self.encode_j_genes(j_genes)
        
        # 7. Convert to numpy arrays
        X_sequences = np.array(X_sequences)
        X_v_genes = np.array(X_v_genes)
        X_j_genes = np.array(X_j_genes)
        y_labels = np.array(labels)
        
        # 8. Create TensorFlow dataset
        dataset = tf.data.Dataset.from_tensor_slices((
            {'cdr3_sequence': X_sequences, 'v_gene': X_v_genes, 'j_gene': X_j_genes,},
            {'labels': y_labels}
        ))
            
        # Map to the format expected by the model: ((inputs), labels)
        dataset = dataset.map(lambda X, y: (X, {'labels': tf.one_hot(y['labels'], len(self.label_map))}))
            
        if shuffle:
            dataset = dataset.shuffle(len(X_sequences), reshuffle_each_iteration=True)
        
        dataset = dataset.batch(batch_size, drop_remainder=True)
        dataset = dataset.prefetch(tf.data.AUTOTUNE)
        
        return dataset
    
    def get_vocab_sizes(self):
        """Return vocabulary sizes for model construction"""
        return {
            'aa_vocab_size': len(self.aa_dict) + 1,  # +1 for padding
            'v_vocab_size': len(self.v_dict) + 1,    # +1 for unknown
            'j_vocab_size': len(self.j_dict) + 1,    # +1 for unknown
            'num_classes': len(self.label_map),
            'max_length': self.max_length
        }
    
    def get_mappings(self):
        """Return the created mappings for inspection"""
        return {
            'aa_dict': self.aa_dict,
            'v_dict': self.v_dict,
            'j_dict': self.j_dict,
            'label_map': self.label_map,
            'column_mapping': self.column_mapping
        }
    
    def get_column_info(self):
        """Get information about detected columns"""
        if self.column_mapping:
            print("Detected column mapping:")
            for standard, actual in self.column_mapping.items():
                print(f"  {standard} -> {actual}")
        else:
            print("No column mapping detected yet. Run load_files() first.")

In [3]:
def create_tcr_model(vocab_sizes):
    """Create a model using the Conv1D architecture from the reference"""
    # Input layers
    cdr3_input = tf.keras.Input(shape=(vocab_sizes['max_length'],), dtype=tf.uint32, name='cdr3_sequence')
    v_gene_input = tf.keras.Input(shape=(), name='v_gene')
    j_gene_input = tf.keras.Input(shape=(), name='j_gene')
    
    # CDR3 sequence encoder (following the reference structure)
    cdr3_embed = tf.keras.layers.Embedding(
        input_dim=vocab_sizes['aa_vocab_size'], 
        output_dim=64, 
        mask_zero=True
    )(cdr3_input)
    
    conv1 = tf.keras.layers.Conv1D(
        filters=64, 
        kernel_size=9, 
        strides=1, 
        activation=tf.keras.activations.relu
    )(cdr3_embed)
    
    conv2 = tf.keras.layers.Conv1D(
        filters=128, 
        kernel_size=7, 
        strides=2, 
        activation=tf.keras.activations.relu
    )(conv1)
    
    conv3 = tf.keras.layers.Conv1D(
        filters=256, 
        kernel_size=5, 
        strides=1, 
        activation=None
    )(conv2)
    
    # Take the first position output (like in reference: [:, 0, :])
    cdr3_encoded = conv3[:, 0, :]
    
    # Gene embeddings
    v_embed = tf.keras.layers.Embedding(vocab_sizes['v_vocab_size'], 32)(v_gene_input)
    j_embed = tf.keras.layers.Embedding(vocab_sizes['j_vocab_size'], 32)(j_gene_input)
    
    # Flatten gene embeddings
    v_flat = tf.keras.layers.Flatten()(v_embed)
    j_flat = tf.keras.layers.Flatten()(j_embed)
    
    # Concatenate all features
    fused = tf.keras.layers.Concatenate()([cdr3_encoded, v_flat, j_flat])
    
    # Classifier (following reference structure)
    dropout1 = tf.keras.layers.Dropout(rate=0.1)(fused)
    dense1 = tf.keras.layers.Dense(units=128, activation=tf.keras.activations.relu)(dropout1)
    dropout2 = tf.keras.layers.Dropout(rate=0.05)(dense1)
    dense2 = tf.keras.layers.Dense(units=64, activation=tf.keras.activations.relu)(dropout2)
    output = tf.keras.layers.Dense(units=vocab_sizes['num_classes'], activation='softmax')(dense2)
    
    model = tf.keras.Model(
        inputs=[cdr3_input, v_gene_input, j_gene_input],
        outputs=output
    )
    
    return model

In [4]:
# Initialize loader
data_dir = 'Data/Human_Antigens'
loader = TCRDataLoader(data_dir)

# Load data
print("Loading data...")
dataset = loader.load_and_encode_data(batch_size=100, shuffle=True)
print("Data loaded successfully!")

Loading data...
Data loaded successfully!


In [5]:
# Create model
vocab_sizes = loader.get_vocab_sizes()
print("Creating model...")
model = create_tcr_model(vocab_sizes)

# Compile model
model.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

# Display model summary
model.summary()

Creating model...


c:\Users\huang\miniconda3\envs\gpu-ml-base\Lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'conv1d' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ cdr3_sequence       │ (None, 23)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 23, 64)    │      1,344 │ cdr3_sequence[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 15, 64)    │     36,928 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 5, 128)    │     57,472 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ v_gene (InputLayer) │ (None)            │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ j_gene (InputLayer) │ (None)            │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 1, 256)    │    164,096 │ conv1d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 32)        │      2,016 │ v_gene[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 32)        │        448 │ j_gene[0][0]      │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 256)       │          0 │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 32)        │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 32)        │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 320)       │          0 │ get_item[0][0],   │
│ (Concatenate)       │                   │            │ flatten[0][0],    │
│                     │                   │            │ flatten_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 320)       │          0 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     41,088 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      8,256 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 7)         │        455 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 312,103 (1.19 MB)

 Trainable params: 312,103 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Check the range of your encoded values BEFORE mapping
# Get raw arrays
df = loader.load_files()
loader.build_vocabulary(df)

# Filter valid data
valid_rows = df['aminoAcid'].notna()
df_valid = df[valid_rows].copy()

# Encode data
sequences = df_valid['aminoAcid'].values
v_genes = df_valid['v_beta'].fillna('UNK').values
j_genes = df_valid['j_beta'].fillna('UNK').values
labels = [loader.label_map[antigen] for antigen in df_valid['Antigen']]

X_sequences = loader.encode_sequences(sequences)
X_v_genes = loader.encode_v_genes(v_genes)
X_j_genes = loader.encode_j_genes(j_genes)

# Convert to numpy
X_sequences = np.array(X_sequences)
X_v_genes = np.array(X_v_genes)
X_j_genes = np.array(X_j_genes)

print("Encoded ranges:")
print(f"  CDR3 min/max: {X_sequences.min()} / {X_sequences.max()}")
print(f"  V gene min/max: {X_v_genes.min()} / {X_v_genes.max()}")
print(f"  J gene min/max: {X_j_genes.min()} / {X_j_genes.max()}")

vocab_sizes = loader.get_vocab_sizes()
print(f"\nModel expects:")
print(f"  CDR3: 0 to {vocab_sizes['aa_vocab_size']-1}")
print(f"  V gene: 0 to {vocab_sizes['v_vocab_size']-1}")
print(f"  J gene: 0 to {vocab_sizes['j_vocab_size']-1}")

Encoded ranges:
  CDR3 min/max: 0 / 20
  V gene min/max: 0 / 59
  J gene min/max: 0 / 13

Model expects:
  CDR3: 0 to 20
  V gene: 0 to 62
  J gene: 0 to 13


In [ ]:
# Train the model
print("Training model...")
history = model.fit(
    dataset, 
    epochs=100,
    verbose=1
)

# Plot training history if you want
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'])
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'])
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.show()

Training model...
Epoch 1/100


c:\Users\huang\miniconda3\envs\gpu-ml-base\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['cdr3_sequence', 'v_gene', 'j_gene']. Received: the structure of inputs={'cdr3_sequence': '*', 'v_gene': '*', 'j_gene': '*'}
  warnings.warn(
c:\Users\huang\miniconda3\envs\gpu-ml-base\Lib\site-packages\keras\src\layers\layer.py:932: UserWarning: Layer 'conv1d' (of type Conv1D) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


IndexError: list index out of range